<a href="https://colab.research.google.com/github/DavoodSZ1993/Dive_into_Deep_Learning/blob/main/5_mpls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 5.1.2 Activation Functions

In [1]:
!pip install d2l==1.0.0-alpha1.post0 --quiet

     |████████████████████████████████| 93 kB 1.2 MB/s 
     |████████████████████████████████| 121 kB 60.3 MB/s 
     |████████████████████████████████| 1.6 MB 47.1 MB/s 
     |████████████████████████████████| 83 kB 2.6 MB/s 


In [2]:
%matplotlib inline
import torch
from d2l import torch as d2l

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
